In [13]:
import pandas as pd
import numpy as np

df_fore = pd.read_csv(r"/Users/jan/Documents/competitions/hackatons/Hacknation2025/data/xgb_forecasts_2025_2027.csv")
df_ratios = pd.read_csv(r"/Users/jan/Documents/competitions/hackatons/Hacknation2025/data/df_ratios_finished_final_8_ratios_jig.csv")

# 1) unify year column name
df_ratios = df_ratios.rename(columns={"rok": "year"})

# 2) list of ratio suffixes from historical file
ratio_cols = [c for c in df_ratios.columns if c != "year"]
prefixes = sorted({c.split("_", 1)[0] for c in ratio_cols})
suffixes = sorted({c.split("_", 1)[1] for c in ratio_cols})

# 3) compute missing NP_to_GS and NP_to_TC in forecasts when possible
for pref in prefixes:
    np_col = f"{pref}_NP"
    gs_col = f"{pref}_GS"
    tc_col = f"{pref}_TC"

    if np_col in df_fore.columns and gs_col in df_fore.columns:
        df_fore[f"{pref}_NP_to_GS"] = df_fore[np_col] / df_fore[gs_col].replace({0: np.nan})

    if np_col in df_fore.columns and tc_col in df_fore.columns:
        df_fore[f"{pref}_NP_to_TC"] = df_fore[np_col] / df_fore[tc_col].replace({0: np.nan})

# 4) build future ratios with exactly the same columns as df_ratios
df_future = df_fore[["year"]].copy()
for c in ratio_cols:
    df_future[c] = df_fore[c] if c in df_fore.columns else np.nan

# 5) stack
df_all = pd.concat([df_ratios, df_future], ignore_index=True)

print(df_all)


    year  10.1_GS_dyn  10.1_NP_dyn  10.1_CF_dyn  10.1_DEBT_to_GS  \
0   2005          NaN          NaN          NaN         0.200806   
1   2006     0.042270    -0.019871     0.042457         0.208385   
2   2007     0.122495     0.227003     0.151375         0.202369   
3   2008     0.044975    -0.635139    -0.047439         0.199708   
4   2009     0.091467     0.724386     0.164312         0.176530   
5   2010     0.009726     1.394732     0.130790         0.180000   
6   2011     0.175451    -0.292150    -0.095271         0.179125   
7   2012     0.136095     0.194771     0.135127         0.169286   
8   2013     0.019817     0.092016     0.016090         0.169305   
9   2014     0.008091     0.575780     0.261256         0.172069   
10  2015     0.010875    -0.009815     0.036379         0.188268   
11  2016     0.076187    -0.130738    -0.042158         0.195434   
12  2017     0.086806     0.144680     0.129474         0.200487   
13  2018    -0.000597     0.284304     0.163967 

/var/folders/pq/224wz8116dq05pr87bp6rkb80000gn/T/ipykernel_88623/1092842648.py:30: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/pq/224wz8116dq05pr87bp6rkb80000gn/T/ipykernel_88623/1092842648.py:30: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/pq/224wz8116dq05pr87bp6rkb80000gn/T/ipykernel_88623/1092842648.py:30: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd

In [14]:


df_ratios = df_ratios.rename(columns={"rok": "year"})
df_fore["year"] = df_fore["year"].astype(int)

# --- identify PKD prefixes / ratio suffixes from historical ---
ratio_cols = [c for c in df_ratios.columns if c != "year"]
prefixes = sorted({c.split("_", 1)[0] for c in ratio_cols})
suffixes = sorted({c.split("_", 1)[1] for c in ratio_cols})

# --- compute missing NP_to_GS / NP_to_TC in forecasts when possible ---
for pref in prefixes:
    np_col = f"{pref}_NP"
    gs_col = f"{pref}_GS"
    tc_col = f"{pref}_TC"

    if f"{pref}_NP_to_GS" not in df_fore.columns and np_col in df_fore.columns and gs_col in df_fore.columns:
        df_fore[f"{pref}_NP_to_GS"] = df_fore[np_col] / df_fore[gs_col].replace({0: np.nan})

    if f"{pref}_NP_to_TC" not in df_fore.columns and np_col in df_fore.columns and tc_col in df_fore.columns:
        df_fore[f"{pref}_NP_to_TC"] = df_fore[np_col] / df_fore[tc_col].replace({0: np.nan})

# --- historical wide -> long -> wide by (year, pkd) ---
long = df_ratios.melt(id_vars=["year"], var_name="pkd_ratio", value_name="value")
long[["pkd", "ratio"]] = long["pkd_ratio"].str.split("_", n=1, expand=True)
long = long.drop(columns=["pkd_ratio"])

wide_hist = (
    long.pivot_table(index=["year", "pkd"], columns="ratio", values="value")
        .reset_index()
)
wide_hist.columns.name = None

# --- build future ratios wide ---
future_rows = []
for _, row in df_fore.iterrows():
    y = int(row["year"])
    for pref in prefixes:
        vals = {}
        ok = False
        for suf in suffixes:
            col = f"{pref}_{suf}"
            if col in df_fore.columns:
                vals[suf] = row[col]
                ok = True
            else:
                vals[suf] = np.nan
        if ok:
            future_rows.append({"year": y, "pkd": pref, **vals})

wide_future = pd.DataFrame(future_rows)

# --- combine ---
wide_all = pd.concat([wide_hist, wide_future], ignore_index=True)

# --- pillar construction ---
wide_all["growth_raw"] = wide_all[["GS_dyn", "NP_dyn", "CF_dyn"]].mean(axis=1, skipna=True)
wide_all["profit_raw"] = wide_all[["NP_to_GS", "NP_to_TC"]].mean(axis=1, skipna=True)
wide_all["cash_raw"]   = wide_all[["CF_to_TC", "CF_dyn"]].mean(axis=1, skipna=True)
wide_all["debt_raw"]   = wide_all[["DEBT_to_GS", "STL_to_GS"]].mean(axis=1, skipna=True)

# --- percentile ranks per year across PKDs ---
def pct_rank(s):
    return s.rank(pct=True)

for col in ["growth_raw", "profit_raw", "cash_raw", "debt_raw"]:
    wide_all[col + "_pct"] = wide_all.groupby("year")[col].transform(pct_rank)

# higher leverage_pct = better balance sheet
wide_all["leverage_pct"] = 1 - wide_all["debt_raw_pct"]

# --- explainable BCG + risk overlay ---
def classify_bcg(row):
    g = row["growth_raw_pct"]
    p = row["profit_raw_pct"]
    c = row["cash_raw_pct"]
    l = row["leverage_pct"]

    # Require these three; allow missing growth for early years
    if pd.isna(p) or pd.isna(c) or pd.isna(l):
        return np.nan
    if pd.isna(g):
        g = 0.5

    g_high = g >= 0.5
    p_high = p >= 0.5

    if g_high and p_high:
        base = "Star"
    elif (not g_high) and p_high:
        base = "Cash cow"
    elif g_high and (not p_high):
        base = "Question mark"
    else:
        base = "Dog"

    # risk tag
    if (c <= 0.33) and (l <= 0.33):
        risk = "high risk"
    elif (c <= 0.33) or (l <= 0.33):
        risk = "elevated risk"
    else:
        risk = "lower risk"

    return f"{base} ({risk})"

wide_all["group"] = wide_all.apply(classify_bcg, axis=1)

# --- summary: 2024 vs predicted 2025-2027 ---
pivot = wide_all.pivot_table(index="pkd", columns="year", values="group", aggfunc="first")
print(pivot[[2024, 2025, 2026, 2027]].sort_index())


year                           2024                           2025  \
pkd                                                                  
10.1          Cash cow (lower risk)     Question mark (lower risk)   
22.2          Cash cow (lower risk)           Star (elevated risk)   
29.3  Question mark (elevated risk)                Dog (high risk)   
35.1            Dog (elevated risk)  Question mark (elevated risk)   
41.2           Cash cow (high risk)           Star (elevated risk)   
45.1            Dog (elevated risk)     Question mark (lower risk)   
46.3     Question mark (lower risk)     Question mark (lower risk)   
46.4              Star (lower risk)       Cash cow (elevated risk)   
46.7            Dog (elevated risk)            Dog (elevated risk)   
46.9              Star (lower risk)          Cash cow (lower risk)   
47.1              Star (lower risk)              Star (lower risk)   
47.4  Question mark (elevated risk)                            NaN   
49.4            Dog 

In [15]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

RATIOS_FILE = r"/Users/jan/Documents/competitions/hackatons/Hacknation2025/data/df_ratios_finished_final_8_ratios_jig.csv"
FORE_FILE   = r"/Users/jan/Documents/competitions/hackatons/Hacknation2025/data/xgb_forecasts_2025_2027.csv"


RATIO_SUFFIXES = [
    "GS_dyn",
    "NP_dyn",
    "CF_dyn",
    "NP_to_GS",
    "DEBT_to_GS",
    "CF_to_TC",
    "STL_to_GS",
    "NP_to_TC",
]

# ---------------------------
# 1) LOAD
# ---------------------------
df_rat = pd.read_csv(RATIOS_FILE)
df_for = pd.read_csv(FORE_FILE)

if "rok" in df_rat.columns:
    df_rat = df_rat.rename(columns={"rok": "year"})

df_rat["year"] = df_rat["year"].astype(int)
df_for["year"] = df_for["year"].astype(int)

ratio_cols = [c for c in df_rat.columns if c != "year"]
pkds = sorted({c.split("_", 1)[0] for c in ratio_cols})

print("Historical shape:", df_rat.shape)
print("Forecast shape:", df_for.shape)
print("PKD count:", len(pkds))
print("Years hist:", df_rat.year.min(), "->", df_rat.year.max())
print("Years fore:", df_for.year.min(), "->", df_for.year.max())

# ---------------------------
# 2) DERIVE missing predicted NP_to_GS and NP_to_TC if needed
# ---------------------------
df_for2 = df_for.copy()

for pkd in pkds:
    np_col = f"{pkd}_NP"
    gs_col = f"{pkd}_GS"
    tc_col = f"{pkd}_TC"

    if (f"{pkd}_NP_to_GS" not in df_for2.columns) and (np_col in df_for2.columns) and (gs_col in df_for2.columns):
        df_for2[f"{pkd}_NP_to_GS"] = df_for2[np_col] / df_for2[gs_col].replace({0: np.nan})

    if (f"{pkd}_NP_to_TC" not in df_for2.columns) and (np_col in df_for2.columns) and (tc_col in df_for2.columns):
        df_for2[f"{pkd}_NP_to_TC"] = df_for2[np_col] / df_for2[tc_col].replace({0: np.nan})

# ---------------------------
# 3) BUILD future ratios wide aligned to historical
# ---------------------------
future_rows = []
for _, row in df_for2.iterrows():
    out = {"year": int(row["year"])}
    for pkd in pkds:
        for suf in RATIO_SUFFIXES:
            col = f"{pkd}_{suf}"
            out[col] = row[col] if col in df_for2.columns else np.nan
    future_rows.append(out)

df_future = pd.DataFrame(future_rows)

df_all_wide = (
    pd.concat([df_rat, df_future], ignore_index=True)
      .sort_values("year")
      .reset_index(drop=True)
)

print("Combined wide shape:", df_all_wide.shape)
display(df_all_wide.head(3))
display(df_all_wide.tail(3))

# ---------------------------
# 4) WIDE -> LONG
# ---------------------------
long = df_all_wide.melt(id_vars=["year"], var_name="pkd_ratio", value_name="value")
long[["pkd", "ratio"]] = long["pkd_ratio"].str.split("_", n=1, expand=True)
long = long.drop(columns=["pkd_ratio"])
long = long[long["ratio"].isin(RATIO_SUFFIXES)].copy()

print("Long shape:", long.shape)
print("Ratios present:", sorted(long["ratio"].unique()))
print("PKDs present:", long["pkd"].nunique())

# ---------------------------
# 5) Year-by-year averages + YoY changes
# ---------------------------
avg_year_ratio = long.groupby(["year", "ratio"], as_index=False)["value"].mean()
avg_pivot = avg_year_ratio.pivot(index="year", columns="ratio", values="value").sort_index()
avg_change = avg_pivot.diff()

print("\n=== Average ratios by year (tail) ===")
display(avg_pivot.tail(8))

print("\n=== YoY change of average ratios (tail) ===")
display(avg_change.tail(8))

# Interactive average lines
avg_long = avg_pivot.reset_index().melt(id_vars="year", var_name="ratio", value_name="avg_value")
fig_avg = px.line(
    avg_long,
    x="year",
    y="avg_value",
    color="ratio",
    markers=True,
    title="Average ratio across PKD branches by year (incl. predictions)"
)
fig_avg.show()

# ---------------------------
# 6) PKD-year table (8 ratios)
# ---------------------------
wide_pkd = (
    long.pivot_table(index=["year", "pkd"], columns="ratio", values="value")
        .reset_index()
)
wide_pkd.columns.name = None

# Indices for matrix
wide_pkd["growth"] = wide_pkd[["GS_dyn", "NP_dyn", "CF_dyn"]].mean(axis=1, skipna=True)
wide_pkd["profit"] = wide_pkd[["NP_to_GS", "NP_to_TC"]].mean(axis=1, skipna=True)
wide_pkd["cash"]   = wide_pkd[["CF_to_TC", "CF_dyn"]].mean(axis=1, skipna=True)
wide_pkd["debt_burden"] = wide_pkd[["DEBT_to_GS", "STL_to_GS"]].mean(axis=1, skipna=True)

display(wide_pkd.head(10))

# ---------------------------
# 7) Explainable yearly matrix classification (median thresholds)
# ---------------------------
def classify_one_year(df):
    g_med = df["growth"].median(skipna=True)
    p_med = df["profit"].median(skipna=True)

    out = df.copy()
    out["g_med"] = g_med
    out["p_med"] = p_med
    out["growth_high"] = out["growth"] >= g_med
    out["profit_high"] = out["profit"] >= p_med

    def base(row):
        if pd.isna(row["growth"]) or pd.isna(row["profit"]):
            return np.nan
        if row["growth_high"] and row["profit_high"]:
            return "Star"
        if (not row["growth_high"]) and row["profit_high"]:
            return "Cash cow"
        if row["growth_high"] and (not row["profit_high"]):
            return "Question mark"
        return "Dog"

    out["group"] = out.apply(base, axis=1)
    return out

wide_pkd = wide_pkd.groupby("year", group_keys=False).apply(classify_one_year)

# Group table (nice for your writeup)
pivot_groups = wide_pkd.pivot_table(index="pkd", columns="year", values="group", aggfunc="first")

print("\n=== Groups for key years ===")
cols = [c for c in [2022, 2023, 2024, 2025, 2026, 2027] if c in pivot_groups.columns]
display(pivot_groups[cols].sort_index())

# ---------------------------
# 8) Animated Growth vs Profit matrix
# ---------------------------
matrix_df = wide_pkd.dropna(subset=["growth", "profit"]).copy()

fig_matrix = px.scatter(
    matrix_df,
    x="growth",
    y="profit",
    color="group",
    animation_frame="year",
    hover_name="pkd",
    hover_data={
        "GS_dyn": True, "NP_dyn": True, "CF_dyn": True,
        "NP_to_GS": True, "NP_to_TC": True,
        "CF_to_TC": True, "DEBT_to_GS": True, "STL_to_GS": True,
        "cash": True, "debt_burden": True,
        "growth": True, "profit": True, "group": True
    },
    title="PKD matrix over time: Growth vs Profit (median-based groups)"
)
fig_matrix.show()

# ---------------------------
# 9) Heatmap of group evolution (compact)
# ---------------------------
years_focus = [2022, 2023, 2024, 2025, 2026, 2027]
heat = pivot_groups.reindex(columns=[y for y in years_focus if y in pivot_groups.columns])

group_order = ["Dog", "Question mark", "Cash cow", "Star"]
group_to_code = {g:i for i,g in enumerate(group_order)}
heat_code = heat.applymap(lambda x: group_to_code.get(x, np.nan))

fig_heat = go.Figure(
    data=go.Heatmap(
        z=heat_code.values,
        x=heat_code.columns.astype(str),
        y=heat_code.index.astype(str),
        customdata=heat.values,
        hovertemplate="PKD %{y}<br>Year %{x}<br>Group: %{customdata}<extra></extra>",
    )
)
fig_heat.update_layout(
    title="Group evolution by PKD (matrix logic)",
    xaxis_title="Year",
    yaxis_title="PKD"
)
fig_heat.show()

Historical shape: (20, 121)
Forecast shape: (3, 533)
PKD count: 15
Years hist: 2005 -> 2024
Years fore: 2025 -> 2027
Combined wide shape: (23, 121)


,year,10.1_GS_dyn,10.1_NP_dyn,10.1_CF_dyn,10.1_DEBT_to_GS,10.1_CF_to_TC,10.1_STL_to_GS,10.1_NP_to_GS,10.1_NP_to_TC,22.2_GS_dyn,...,52.2_NP_to_GS,52.2_NP_to_TC,62.0_GS_dyn,62.0_NP_dyn,62.0_CF_dyn,62.0_DEBT_to_GS,62.0_CF_to_TC,62.0_STL_to_GS,62.0_NP_to_GS,62.0_NP_to_TC
0,2005,NaN,NaN,NaN,0.200806,0.048337,0.133217,0.019857,0.020330,NaN,...,0.050816,0.054346,NaN,NaN,NaN,0.344450,0.142564,0.290866,0.075210,0.082341
1,2006,0.042270,-0.019871,0.042457,0.208385,0.048288,0.141798,0.018673,0.019096,0.140223,...,0.056970,0.061527,-0.009348,-0.089460,-0.030471,0.323346,0.139568,0.277943,0.069128,0.075706
2,2007,0.122495,0.227003,0.151375,0.202369,0.049630,0.146417,0.020411,0.020915,0.187502,...,0.072333,0.079331,0.146006,0.209094,0.068826,0.336678,0.130897,0.271881,0.072933,0.080321


,year,10.1_GS_dyn,10.1_NP_dyn,10.1_CF_dyn,10.1_DEBT_to_GS,10.1_CF_to_TC,10.1_STL_to_GS,10.1_NP_to_GS,10.1_NP_to_TC,22.2_GS_dyn,...,52.2_NP_to_GS,52.2_NP_to_TC,62.0_GS_dyn,62.0_NP_dyn,62.0_CF_dyn,62.0_DEBT_to_GS,62.0_CF_to_TC,62.0_STL_to_GS,62.0_NP_to_GS,62.0_NP_to_TC
20,2025,0.072049,0.370391,0.266829,0.191901,0.061997,0.141275,0.033520,0.034931,0.073055,...,0.052313,0.056172,0.301479,0.325092,0.296742,0.23128,0.114316,0.193327,0.073967,0.081286
21,2026,0.072049,0.370391,0.266829,0.191901,0.061997,0.141275,0.033520,0.034931,0.073055,...,0.052313,0.056172,0.301479,0.325092,0.296742,0.23128,0.114316,0.193327,0.073967,0.081286
22,2027,0.321512,0.404271,0.333252,0.178992,0.061997,0.134680,0.033701,0.035177,0.073055,...,0.052313,0.056172,0.301479,0.325092,0.296742,0.23128,0.114316,0.193327,0.073967,0.081286


Long shape: (2760, 4)
Ratios present: ['CF_dyn', 'CF_to_TC', 'DEBT_to_GS', 'GS_dyn', 'NP_dyn', 'NP_to_GS', 'NP_to_TC', 'STL_to_GS']
PKDs present: 15

=== Average ratios by year (tail) ===


ratio,CF_dyn,CF_to_TC,DEBT_to_GS,GS_dyn,NP_dyn,NP_to_GS,NP_to_TC,STL_to_GS
year,,,,,,,,
2020,0.127146,0.075514,0.345202,0.007574,0.088048,0.033633,0.035901,0.226289
2021,0.293281,0.081196,0.329061,0.173669,-0.656984,0.045298,0.048261,0.228117
2022,0.212184,0.079589,0.299039,0.219772,0.147795,0.042457,0.045192,0.213930
2023,0.048152,0.074678,0.285974,0.101309,-0.105592,0.035296,0.037413,0.205883
2024,-0.048439,0.071670,0.299497,-0.001138,-0.258673,0.032440,0.034027,0.213290
2025,0.190699,0.078686,0.327921,0.119761,-0.082905,0.043009,0.045760,0.222185
2026,0.190699,0.078686,0.327921,0.119761,-0.082905,0.043009,0.045760,0.222185
2027,0.184496,0.078199,0.327730,0.136106,-0.090261,0.043022,0.045777,0.222284


...

,year,pkd,CF_dyn,CF_to_TC,DEBT_to_GS,GS_dyn,NP_dyn,NP_to_GS,NP_to_TC,STL_to_GS,growth,profit,cash,debt_burden
0,2005,10.1,NaN,0.048337,0.200806,NaN,NaN,0.019857,0.020330,0.133217,NaN,0.020094,0.048337,0.167011
1,2005,22.2,NaN,0.109627,0.320738,NaN,NaN,0.044657,0.047179,0.244635,NaN,0.045918,0.109627,0.282686
2,2005,29.3,NaN,0.122344,0.343810,NaN,NaN,0.057006,0.061128,0.254081,NaN,0.059067,0.122344,0.298946
3,2005,35.1,NaN,0.116149,0.310608,NaN,NaN,0.034853,0.036543,0.157711,NaN,0.035698,0.116149,0.234160
4,2005,41.2,NaN,0.046796,0.343127,NaN,NaN,0.020894,0.021485,0.305324,NaN,0.021189,0.046796,0.324226
5,2005,45.1,NaN,0.027444,0.220796,NaN,NaN,0.006152,0.006206,0.194773,NaN,0.006179,0.027444,0.207785
6,2005,46.3,NaN,0.028563,0.177796,NaN,NaN,0.014630,0.014899,0.142218,NaN,0.014765,0.028563,0.160007
7,2005,46.4,NaN,0.041702,0.248753,NaN,NaN,0.024544,0.025305,0.229867,NaN,0.024925,0.041702,0.239310
8,2005,46.7,NaN,0.035427,0.188985,NaN,NaN,0.016245,0.016586,0.156380,NaN,0.016416,0.035427,0.172682
9,2005,46.9,NaN,0.058555,0.239305,NaN,NaN,0.035209,0.036775,0.205601,NaN,0.035992,0.058555,0.222453



=== Groups for key years ===


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:1214: RuntimeWarning:

Mean of empty slice

/var/folders/pq/224wz8116dq05pr87bp6rkb80000gn/T/ipykernel_88623/2735335285.py:163: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



year,2022,2023,2024,2025,2026,2027
pkd,,,,,,
10.1,Star,Star,Cash cow,Question mark,Question mark,Question mark
22.2,Cash cow,Cash cow,Cash cow,Star,Star,Star
29.3,Question mark,Question mark,Question mark,Dog,Dog,Dog
35.1,Dog,Dog,Dog,Question mark,Question mark,Question mark
41.2,Star,Cash cow,Cash cow,Star,Star,Star
45.1,Question mark,Question mark,Dog,Dog,Dog,Dog
46.3,Dog,Question mark,Question mark,Question mark,Question mark,Question mark
46.4,Dog,Star,Star,Cash cow,Cash cow,Cash cow
46.7,Cash cow,Dog,Dog,Dog,Dog,Dog


/var/folders/pq/224wz8116dq05pr87bp6rkb80000gn/T/ipykernel_88623/2735335285.py:203: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.



In [16]:
import plotly.express as px

matrix_df = wide_pkd.dropna(subset=["growth", "profit"]).copy()

# FIXED AXIS RANGES
g_lo, g_hi = -3, 3
p_lo, p_hi = -0.1, 0.2

group_order = ["Dog", "Question mark", "Cash cow", "Star"]
color_map = {
    "Dog": "#6b7280",
    "Question mark": "#f59e0b",
    "Cash cow": "#10b981",
    "Star": "#3b82f6",
}

fig = px.scatter(
    matrix_df,
    x="growth",
    y="profit",
    color="group",
    text="pkd",
    animation_frame="year",
    animation_group="pkd",
    category_orders={"group": group_order},
    color_discrete_map=color_map,
    hover_name="pkd",
    hover_data={
        "GS_dyn": True, "NP_dyn": True, "CF_dyn": True,
        "NP_to_GS": True, "NP_to_TC": True,
        "CF_to_TC": True, "DEBT_to_GS": True, "STL_to_GS": True,
        "cash": True, "debt_burden": True,
        "growth": True, "profit": True,
    },
    title="PKD matrix over time: Growth vs Profit",
)

fig.update_traces(
    textposition="top center",
    textfont_size=10,
    marker=dict(size=16, opacity=0.85, line=dict(width=1, color="black"))
)

# lock axes
fig.update_xaxes(range=[g_lo, g_hi], zeroline=True)
fig.update_yaxes(range=[p_lo, p_hi], zeroline=True)

# medians (global)
g_med = matrix_df["growth"].median()
p_med = matrix_df["profit"].median()

fig.add_vline(x=g_med, line_dash="dot", opacity=0.6)
fig.add_hline(y=p_med, line_dash="dot", opacity=0.6)

# --- quadrant shading using FIXED ranges ---
fig.add_vrect(x0=g_lo, x1=g_med, fillcolor="rgba(107,114,128,0.08)", line_width=0)
fig.add_vrect(x0=g_med, x1=g_hi, fillcolor="rgba(59,130,246,0.06)", line_width=0)

fig.add_hrect(y0=p_lo, y1=p_med, fillcolor="rgba(107,114,128,0.05)", line_width=0)
fig.add_hrect(y0=p_med, y1=p_hi, fillcolor="rgba(16,185,129,0.05)", line_width=0)

# --- annotation positions INSIDE range ---
y_top = p_lo + 0.85*(p_hi - p_lo)
y_bot = p_lo + 0.15*(p_hi - p_lo)

# Big quadrant labels
fig.add_annotation(x=g_lo, y=y_bot, text="Dog zone", showarrow=False, xanchor="left", font=dict(size=12))
fig.add_annotation(x=g_hi, y=y_bot, text="Question mark zone", showarrow=False, xanchor="right", font=dict(size=12))
fig.add_annotation(x=g_lo, y=y_top, text="Cash cow zone", showarrow=False, xanchor="left", font=dict(size=12))
fig.add_annotation(x=g_hi, y=y_top, text="Star zone", showarrow=False, xanchor="right", font=dict(size=12))

# optional: reduce label clutter
fig.update_layout(uniformtext_minsize=8, uniformtext_mode="hide")

# smoother animation
fig.update_layout(transition={"duration": 500})

fig.show()


In [20]:
import pandas as pd
import numpy as np
import plotly.express as px
from IPython.display import display

# Zakładam, że masz już wide_pkd z poprzedniego pipeline'u, tzn.:
# columns: year, pkd, GS_dyn, NP_dyn, CF_dyn, NP_to_GS, NP_to_TC, CF_to_TC, DEBT_to_GS, STL_to_GS
# + ewentualnie już wyliczone growth/profit/cash/debt_burden
# Jeśli nie, odkomentuj poniższy blok:

# wide_pkd["growth"] = wide_pkd[["GS_dyn", "NP_dyn", "CF_dyn"]].mean(axis=1, skipna=True)
# wide_pkd["profit"] = wide_pkd[["NP_to_GS", "NP_to_TC"]].mean(axis=1, skipna=True)
# wide_pkd["cash"]   = wide_pkd[["CF_to_TC", "CF_dyn"]].mean(axis=1, skipna=True)
# wide_pkd["debt_burden"] = wide_pkd[["DEBT_to_GS", "STL_to_GS"]].mean(axis=1, skipna=True)

# --- 4 FILARY (czytelne i zgodne z opisem) ---
wide_pkd["growth"] = wide_pkd[["GS_dyn", "NP_dyn", "CF_dyn"]].mean(axis=1, skipna=True)
wide_pkd["profit"] = wide_pkd[["NP_to_GS", "NP_to_TC"]].mean(axis=1, skipna=True)

# cash: uwzględniamy pokrycie kosztów cash flow i dynamikę CF
wide_pkd["cash"] = wide_pkd[["CF_to_TC", "CF_dyn"]].mean(axis=1, skipna=True)

# debt burden: niżej = lepiej
wide_pkd["debt_burden"] = wide_pkd[["DEBT_to_GS", "STL_to_GS"]].mean(axis=1, skipna=True)

# --- Z-SCORE per rok (klucz do porównań w czasie) ---
def zscore(s):
    sd = s.std(ddof=0)
    if sd == 0 or np.isnan(sd):
        return (s - s.mean()) * 0
    return (s - s.mean()) / sd

for col in ["growth", "profit", "cash", "debt_burden"]:
    wide_pkd[col + "_z"] = wide_pkd.groupby("year")[col].transform(zscore)

# odwracamy zadłużenie, żeby "więcej = lepiej"
wide_pkd["leverage_z"] = -wide_pkd["debt_burden_z"]

# --- WAGI (jawne i łatwe do obrony) ---
W_GROWTH = 0.35
W_PROFIT = 0.35
W_CASH   = 0.20
W_LEV    = 0.10

# --- IKB: Indeks Kondycji Branży ---
wide_pkd["IKB"] = (
    W_GROWTH * wide_pkd["growth_z"] +
    W_PROFIT * wide_pkd["profit_z"] +
    W_CASH   * wide_pkd["cash_z"] +
    W_LEV    * wide_pkd["leverage_z"]
)

# --- FPI: presja finansowania (intuicyjna miara potrzeb pożyczkowych) ---
# wzrost zwiększa potrzeby, słaby cash i wysokie zadłużenie też:
wide_pkd["FPI"] = (
    wide_pkd["growth_z"]
    - wide_pkd["cash_z"]
    - wide_pkd["leverage_z"]
)

print("Podgląd filarów i indeksów:")
display(
    wide_pkd.loc[:, [
        "year","pkd",
        "growth","profit","cash","debt_burden",
        "growth_z","profit_z","cash_z","leverage_z",
        "IKB","FPI"
    ]].sort_values(["year","pkd"]).head(20)
)


Podgląd filarów i indeksów:


,year,pkd,growth,profit,cash,debt_burden,growth_z,profit_z,cash_z,leverage_z,IKB,FPI
0,2005,10.1,NaN,0.020094,0.048337,0.167011,NaN,-0.522051,-0.559304,1.298936,NaN,NaN
1,2005,22.2,NaN,0.045918,0.109627,0.282686,NaN,0.746420,0.944325,-0.777483,NaN,NaN
2,2005,29.3,NaN,0.059067,0.122344,0.298946,NaN,1.392262,1.256295,-1.069344,NaN,NaN
3,2005,35.1,NaN,0.035698,0.116149,0.234160,NaN,0.244399,1.104323,0.093593,NaN,NaN
4,2005,41.2,NaN,0.021189,0.046796,0.324226,NaN,-0.468221,-0.597108,-1.523132,NaN,NaN
5,2005,45.1,NaN,0.006179,0.027444,0.207785,NaN,-1.205519,-1.071871,0.567033,NaN,NaN
6,2005,46.3,NaN,0.014765,0.028563,0.160007,NaN,-0.783796,-1.044417,1.424664,NaN,NaN
7,2005,46.4,NaN,0.024925,0.041702,0.239310,NaN,-0.284757,-0.722089,0.001144,NaN,NaN
8,2005,46.7,NaN,0.016416,0.035427,0.172682,NaN,-0.702706,-0.876016,1.197134,NaN,NaN
9,2005,46.9,NaN,0.035992,0.058555,0.222453,NaN,0.258840,-0.308633,0.303733,NaN,NaN


In [21]:
ikb_rank = wide_pkd.dropna(subset=["IKB"]).copy()

fig_rank = px.bar(
    ikb_rank.sort_values(["year","IKB"]),
    x="IKB",
    y="pkd",
    orientation="h",
    animation_frame="year",
    range_x=[
        ikb_rank["IKB"].quantile(0.02),
        ikb_rank["IKB"].quantile(0.98)
    ],
    title="Indeks Kondycji Branży (IKB) – ranking PKD w czasie",
)

fig_rank.update_layout(yaxis={"categoryorder":"total ascending"})
fig_rank.show()


In [22]:
ikb_line = wide_pkd.dropna(subset=["IKB"]).copy().sort_values(["pkd","year"])

fig_ikb = px.line(
    ikb_line,
    x="year",
    y="IKB",
    color="pkd",
    markers=True,
    title="IKB – trendy PKD w czasie (historia + prognozy)"
)
fig_ikb.show()


In [23]:
pkd_pick = "46.4"  # zmień na dowolny kod

d = wide_pkd[wide_pkd["pkd"] == pkd_pick].sort_values("year")

display(d[["year","pkd","growth","profit","cash","debt_burden","IKB","FPI"]])

fig_pillars = px.line(
    d,
    x="year",
    y=["growth_z","profit_z","cash_z","leverage_z","IKB","FPI"],
    markers=True,
    title=f"PKD {pkd_pick} – filary (z-score) + IKB i FPI"
)
fig_pillars.show()


,year,pkd,growth,profit,cash,debt_burden,IKB,FPI
7,2005,46.4,NaN,0.024925,0.041702,0.239310,NaN,NaN
22,2006,46.4,0.097780,0.027253,0.054526,0.236512,-0.258069,1.043750
37,2007,46.4,0.188213,0.030011,0.140618,0.248915,-0.388605,0.019272
52,2008,46.4,-0.026939,0.023390,0.044920,0.267711,-0.197500,0.713971
67,2009,46.4,-0.116098,0.016978,-0.068194,0.254041,-0.782732,0.619313
82,2010,46.4,0.306026,0.028806,0.148031,0.254360,-0.068411,0.101804
97,2011,46.4,-0.114173,0.018652,-0.014722,0.265160,-0.485037,0.186569
112,2012,46.4,0.033479,0.020190,0.014877,0.269427,0.033866,0.514188
127,2013,46.4,0.249404,0.028518,0.134225,0.270097,0.128026,0.008744
142,2014,46.4,0.183249,0.033055,0.167389,0.276019,0.270262,-0.058690
